<a href="https://colab.research.google.com/github/lulu0o6/Super-Literature-Search/blob/main/%E2%80%9C3Dec2025%E6%96%87%E7%8C%AE%E6%90%9C%E7%B4%A2V1_1_%E6%AD%A3%E5%BC%8F%E7%89%88%EF%BC%88%E5%A2%9E%E5%8A%A0%E4%BD%9C%E8%80%85%E6%90%9C%E7%B4%A2%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.安装库

In [72]:
# 单元格1：安装必要的库
print("📦 安装必要的库...")
!pip install requests beautifulsoup4 googletrans==4.0.0-rc1 tqdm

import requests, re, datetime, urllib.parse, time, os, json
from bs4 import BeautifulSoup
from googletrans import Translator
from tqdm import tqdm
from difflib import SequenceMatcher

print("✅ 所有库安装和导入完成！")

📦 安装必要的库...
✅ 所有库安装和导入完成！


# 2 配置搜索函数

In [73]:
# 首先定义缺失的变量
MAX_RESULTS_PER_SOURCE = 50  # 每个来源的最大结果数 (用户请求)
ENTREZ_EMAIL = "your.email@example.com" # 请替换为您的邮箱，PubMed API需要

# ---------- 搜索配置 ----------
KEYWORDS = ["GLP-1", "depression", "mood"] # 用户期望的关键词
AUTHORS = []  # 作者列表
SEARCH_BY_AUTHOR = False  # 是否启用作者搜索，默认为False
SEARCH_LOGIC = "or"  # 用户期望的搜索逻辑
START_YEAR = 2000  # 用户期望的开始年份
END_YEAR = 2025 # 用户期望的结束年份
LOGIC_OPERATOR = "CUSTOM"  # 设置为CUSTOM模式

def build_custom_query(api_type="general"):
    """构建自定义搜索逻辑查询"""
    base_query = ""

    # 根据 SEARCH_LOGIC 构建基础查询部分
    if SEARCH_LOGIC.lower() == "or":
        # 对于 PubMed，使用 [Title/Abstract] 字段
        if api_type == "pubmed":
            base_query_terms = [f'"{k}"[Title/Abstract]' for k in KEYWORDS]
            base_query = " OR ".join(base_query_terms)
        # 对于 CrossRef 和 Semantic Scholar，不需要引号和字段限制
        elif api_type in ["crossref", "semantic"]:
            base_query = " OR ".join(KEYWORDS)
        # 默认情况
        else:
            base_query_terms = [f'"{k}"' for k in KEYWORDS]
            base_query = " OR ".join(base_query_terms)
    elif SEARCH_LOGIC.lower() == "and":
        # 对于 PubMed，使用 [Title/Abstract] 字段
        if api_type == "pubmed":
            base_query_terms = [f'"{k}"[Title/Abstract]' for k in KEYWORDS]
            base_query = " AND ".join(base_query_terms)
        # 对于 CrossRef 和 Semantic Scholar，不需要引号和字段限制
        elif api_type in ["crossref", "semantic"]:
            base_query = " AND ".join(KEYWORDS)
        # 默认情况
        else:
            base_query_terms = [f'"{k}"' for k in KEYWORDS]
            base_query = " AND ".join(base_query_terms)
    # 如果是自定义逻辑字符串
    elif LOGIC_OPERATOR == "CUSTOM":
        if api_type == "pubmed":
            # 将关键词替换为 PubMed 格式
            pubmed_search_logic = SEARCH_LOGIC
            for keyword in KEYWORDS:
                pubmed_search_logic = pubmed_search_logic.replace(keyword, f'"{keyword}"[Title/Abstract]')
            base_query = pubmed_search_logic
        elif api_type in ["crossref", "semantic"]:
            base_query = SEARCH_LOGIC
        else:
            base_query = SEARCH_LOGIC

    # 添加作者搜索
    if SEARCH_BY_AUTHOR and AUTHORS:
        if api_type == "pubmed":
            author_query = " OR ".join([f'"{author}"[Author]' for author in AUTHORS])
            base_query = f"({base_query}) AND ({author_query})"
        elif api_type == "semantic":
            author_query = " OR ".join(AUTHORS)
            base_query = f"({base_query}) AND author:({author_query})"
        else:
            author_query = " OR ".join(AUTHORS)
            base_query = f"({base_query}) AND ({author_query})"

    return base_query

# 日期格式
from datetime import datetime
now = datetime.now()
DATE_PREFIX = f"{now.day}{now.strftime('%b%Y')}" # 更改日期格式为 DayMonthYear

from googletrans import Translator
translator = Translator()

print("⚙️ 配置搜索参数...")
print(f"🔍 关键词: {KEYWORDS}")
print(f"👤 作者: {AUTHORS if AUTHORS else '未指定'}")
print(f"🎯 搜索逻辑: {SEARCH_LOGIC}")
print(f"📅 时间范围: {START_YEAR}-{END_YEAR}")
print(f"📊 搜索模式: {LOGIC_OPERATOR}")
print(f"🔎 作者搜索: {'启用' if SEARCH_BY_AUTHOR else '禁用'}")
print(f"📁 日期前缀: {DATE_PREFIX}")
print(f"📊 每源最大结果数: {MAX_RESULTS_PER_SOURCE}")
print("✅ 搜索配置完成!")

⚙️ 配置搜索参数...
🔍 关键词: ['GLP-1', 'depression', 'mood']
👤 作者: 未指定
🎯 搜索逻辑: or
📅 时间范围: 2000-2025
📊 搜索模式: CUSTOM
🔎 作者搜索: 禁用
📁 日期前缀: 3Dec2025
📊 每源最大结果数: 50
✅ 搜索配置完成!


# 3.影响因子期刊数据库

In [74]:
# 单元格3：期刊影响因子数据库
print("📚 加载期刊影响因子数据库...")

JOURNAL_IMPACT_FACTORS = {
    "Nature": 64.8, "Science": 56.9, "Cell": 66.8, "Lancet": 202.7,
    "New England Journal of Medicine": 176.1, "JAMA": 157.3,
    "Nature Medicine": 87.2, "Nature Reviews Drug Discovery": 112.3,
    "Molecular Psychiatry": 15.9, "American Journal of Psychiatry": 18.3,
    "Biological Psychiatry": 12.1, "Journal of Clinical Psychiatry": 6.5,
    "Neuropsychopharmacology": 7.8, "Brain": 15.3, "Neuron": 18.7,
    "Diabetes Care": 19.1, "Journal of Clinical Endocrinology & Metabolism": 6.1,
    "Diabetes": 9.5, "Molecular Metabolism": 8.6,
    "Nature Neuroscience": 28.7, "Nature Reviews Neuroscience": 38.8,
    "Journal of Neuroscience": 6.7, "PLOS ONE": 3.7,
    "Proceedings of the National Academy of Sciences": 11.1,
    "Journal of Biological Chemistry": 5.5, "Scientific Reports": 4.6,
}

def get_journal_impact_factor(journal_name):
    """获取期刊影响因子"""
    if not journal_name:
        return 0
    journal_lower = journal_name.lower()

    # 精确匹配
    for journal, impact_factor in JOURNAL_IMPACT_FACTORS.items():
        if journal.lower() in journal_lower or journal_lower in journal.lower():
            return impact_factor

    # 模糊匹配
    for journal, impact_factor in JOURNAL_IMPACT_FACTORS.items():
        journal_words = set(journal.lower().split())
        input_words = set(journal_name.lower().split())
        common_words = journal_words & input_words
        if len(common_words) >= 2:
            return impact_factor

    return 0

print(f"✅ 已加载 {len(JOURNAL_IMPACT_FACTORS)} 种期刊的影响因子数据")

📚 加载期刊影响因子数据库...
✅ 已加载 26 种期刊的影响因子数据


In [75]:
# 单元格3：期刊影响因子数据库
print("📚 加载期刊影响因子数据库...")

JOURNAL_IMPACT_FACTORS = {
    "Nature": 64.8, "Science": 56.9, "Cell": 66.8, "Lancet": 202.7,
    "New England Journal of Medicine": 176.1, "JAMA": 157.3,
    "Nature Medicine": 87.2, "Nature Reviews Drug Discovery": 112.3,
    "Molecular Psychiatry": 15.9, "American Journal of Psychiatry": 18.3,
    "Biological Psychiatry": 12.1, "Journal of Clinical Psychiatry": 6.5,
    "Neuropsychopharmacology": 7.8, "Brain": 15.3, "Neuron": 18.7,
    "Diabetes Care": 19.1, "Journal of Clinical Endocrinology & Metabolism": 6.1,
    "Diabetes": 9.5, "Molecular Metabolism": 8.6,
    "Nature Neuroscience": 28.7, "Nature Reviews Neuroscience": 38.8,
    "Journal of Neuroscience": 6.7, "PLOS ONE": 3.7,
    "Proceedings of the National Academy of Sciences": 11.1,
    "Journal of Biological Chemistry": 5.5, "Scientific Reports": 4.6,
}

def get_journal_impact_factor(journal_name):
    """获取期刊影响因子"""
    if not journal_name:
        return 0
    journal_lower = journal_name.lower()

    # 精确匹配
    for journal, impact_factor in JOURNAL_IMPACT_FACTORS.items():
        if journal.lower() in journal_lower or journal_lower in journal.lower():
            return impact_factor

    # 模糊匹配
    for journal, impact_factor in JOURNAL_IMPACT_FACTORS.items():
        journal_words = set(journal.lower().split())
        input_words = set(journal_name.lower().split())
        common_words = journal_words & input_words
        if len(common_words) >= 2:
            return impact_factor

    return 0

print(f"✅ 已加载 {len(JOURNAL_IMPACT_FACTORS)} 种期刊的影响因子数据")

📚 加载期刊影响因子数据库...
✅ 已加载 26 种期刊的影响因子数据


# 4.加载工具函数

In [76]:
# 单元格4：工具函数
print("🛠️ 加载工具函数...")

def clean_html(text):
    """清理HTML标签"""
    if not text:
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def translate_to_cn(text):
    """翻译英文到中文"""
    try:
        if not text or not text.strip():
            return ""
        translated = translator.translate(text, src='en', dest='zh-cn').text
        return translated
    except Exception as e:
        print(f"⚠️ 翻译失败: {e}")
        return ""

def doi_to_link(doi):
    """将DOI转换为链接"""
    return f"https://doi.org/{doi}" if doi else ""

def get_citation_count(title, authors, doi):
    """获取引用次数"""
    try:
        if doi:
            # 尝试Semantic Scholar
            url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=citationCount"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return data.get('citationCount', 0)
    except:
        pass
    return 0

print("✅ 工具函数加载完成！")

🛠️ 加载工具函数...
✅ 工具函数加载完成！


# 5.去重功能

In [77]:
# 单元格5：去重功能
print("🔄 加载去重功能...")

def normalize_text(text):
    """标准化文本用于比较"""
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def calculate_similarity(text1, text2):
    """计算两个文本的相似度"""
    norm1 = normalize_text(text1)
    norm2 = normalize_text(text2)

    if not norm1 or not norm2:
        return 0

    return SequenceMatcher(None, norm1, norm2).ratio()

def is_duplicate(item1, item2, similarity_threshold=0.85):
    """判断两个文献是否重复"""
    if item1["DOI"] and item2["DOI"] and item1["DOI"] == item2["DOI"]:
        return True

    if item1["Link"] and item2["Link"] and item1["Link"] == item2["Link"]:
        return True

    title_similarity = calculate_similarity(item1["Title"], item2["Title"])
    if title_similarity > similarity_threshold:
        return True

    return False

def advanced_deduplication(items):
    """高级去重算法"""
    print(f"\n🧩 对 {len(items)} 个项目进行高级去重...")

    unique_items = []
    seen_keys = set()

    for item in items:
        doi_key = item["DOI"] if item["DOI"] else ""
        title_key = normalize_text(item["Title"])[:100] if item["Title"] else ""
        author_year_key = f"{normalize_text(item['Authors'])[:50]}_{item['Year']}" if item["Authors"] else ""

        if doi_key:
            key = doi_key
        elif title_key and author_year_key:
            key = f"{title_key}_{author_year_key}"
        else:
            key = title_key

        if key and key not in seen_keys:
            seen_keys.add(key)
            unique_items.append(item)

    print(f"📊 去重后剩余: {len(unique_items)} 个项目")
    return unique_items

print("✅ 去重功能加载完成！")

🔄 加载去重功能...
✅ 去重功能加载完成！


# 6.摘要补全

In [78]:
# 单元格6：优化后的摘要获取功能 - 精简高效版
print("📄 加载高效版摘要获取功能...")

import re
import requests
import difflib
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time

def is_valid_research_paper(item):
    """检查是否为有效的研究论文（过滤掉评论、决策信等）"""
    title = item.get("Title", "").lower()

    # 排除非研究论文的类型
    invalid_types = [
        "review for", "decision letter for", "response to",
        "revision", "editorial", "comment", "letter to editor"
    ]

    if any(invalid_type in title for invalid_type in invalid_types):
        return False

    # 检查DOI格式（排除明显无效的DOI）
    doi = item.get("DOI", "")
    if doi and any(x in doi for x in ["/review", "/decision", "/v1/", "/v2/", "/v3/"]):
        return False

    return True

def smart_abstract_extraction(soup):
    """智能摘要提取 - 高效版"""
    # 移除无关元素
    for script in soup(["script", "style", "nav", "header", "footer", "aside", "button", "meta", "link"]):
        script.decompose()

    # 核心选择器（经过验证有效的）
    high_priority_selectors = [
        'div.abstract', 'section.abstract', '#abstract',
        'div.abstract-content', 'div[class*="abstract"]',
        'div.c-article-section__content', 'section#Abstract',
        'div.abstract-section', '.article-abstract',
        'div.summary', 'section.summary', '#summary',
    ]

    # 第一轮：精确匹配选择器
    for selector in high_priority_selectors:
        try:
            elements = soup.select(selector)
            for element in elements:
                text = element.get_text(strip=True, separator=' ')
                text = clean_html(text)
                if is_valid_abstract(text):
                    return text
        except Exception:
            continue

    # 第二轮：meta-description提取
    meta_desc = soup.find("meta", attrs={"name": "description"})
    if meta_desc and meta_desc.get("content"):
        text = clean_html(meta_desc["content"])
        if is_valid_abstract(text):
            return text

    # 第三轮：查找包含摘要关键词的段落
    abstract_keywords = ['abstract', 'summary', '概述', '摘要']
    for keyword in abstract_keywords:
        elements = soup.find_all(string=re.compile(keyword, re.I))
        for element in elements:
            parent = element.parent
            if parent and parent.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                next_elem = parent.find_next_sibling()
                if next_elem and next_elem.name == 'p':
                    text = next_elem.get_text(strip=True, separator=' ')
                    text = clean_html(text)
                    if is_valid_abstract(text):
                        return text

    return ""

def is_valid_abstract(text):
    """检查是否为有效的摘要文本"""
    if not text:
        return False

    text_lower = text.lower()

    # 排除明显不是摘要的内容
    invalid_indicators = [
        'login', 'sign in', 'register', 'download', 'citation',
        'references', 'copyright', '©', 'all rights reserved',
        'doi:', 'published by', 'received', 'accepted', 'page',
        'volume', 'issue', 'pp.', 'pp ', 'vol.', 'no.', 'article'
    ]

    if any(indicator in text_lower for indicator in invalid_indicators):
        return False

    # 检查长度和内容质量
    word_count = len(text.split())
    return (80 <= len(text) <= 3000 and
            10 <= word_count <= 500 and
            not text.isupper())

def extract_abstract_from_doi_webpage(doi):
    """从DOI网页提取摘要 - 高效版"""
    if not doi:
        return ""

    try:
        doi_url = f"https://doi.org/{doi}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        }

        response = requests.get(doi_url, headers=headers, timeout=10)

        # 如果DOI页面重定向或内容过少，尝试获取实际文章页面
        if response.status_code != 200 or len(response.content) < 2000:
            pub_url = get_publisher_url_from_crossref(doi)
            if pub_url:
                response = requests.get(pub_url, headers=headers, timeout=10)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            abstract = smart_abstract_extraction(soup)
            return abstract

    except Exception as e:
        pass  # 静默失败，继续其他方法

    return ""

def get_publisher_url_from_crossref(doi):
    """通过CrossRef获取publisher链接"""
    try:
        cr_api = f"https://api.crossref.org/works/{doi}"
        headers = {'User-Agent': 'AcademicResearchBot/1.0'}
        cr_resp = requests.get(cr_api, headers=headers, timeout=8)

        if cr_resp.status_code == 200:
            cr_data = cr_resp.json()
            # 尝试获取文章链接
            url = cr_data.get("message", {}).get("URL")
            if url:
                return url

            # 尝试链接列表
            links = cr_data.get("message", {}).get("link", [])
            for link in links:
                if link.get("content-type") == "text/html":
                    return link.get("URL")

    except Exception:
        pass

    return None

def get_abstract_from_apis(doi):
    """通过API获取摘要 - 高效版"""
    if not doi:
        return ""

    # 按成功率排序的API列表
    apis = [
        (f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=abstract",
         lambda data: data.get('abstract'),
         "Semantic Scholar"),

        (f"https://api.crossref.org/works/{doi}",
         lambda data: data.get('message', {}).get('abstract'),
         "CrossRef"),

        (f"https://api.openalex.org/works/https://doi.org/{doi}",
         lambda data: convert_inverted_index_to_text(data.get('abstract_inverted_index')),
         "OpenAlex"),
    ]

    for api_url, extract_func, api_name in apis:
        try:
            headers = {'User-Agent': 'AcademicResearchBot/1.0'}
            response = requests.get(api_url, headers=headers, timeout=8)

            if response.status_code == 200:
                data = response.json()
                abstract = extract_func(data)

                if abstract and is_valid_abstract(abstract):
                    abstract_text = clean_html(abstract)
                    return abstract_text
        except Exception:
            continue

    return ""

def get_abstract_from_pubmed_by_doi(doi):
    """通过PubMed API获取摘要"""
    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        search_params = {
            'db': 'pubmed',
            'term': f'{doi}[DOI]',
            'retmode': 'json',
            'retmax': 1
        }

        headers = {'User-Agent': 'AcademicResearchBot/1.0'}

        search_response = requests.get(base_url, params=search_params, headers=headers, timeout=8)
        if search_response.status_code == 200:
            search_data = search_response.json()
            pmids = search_data.get('esearchresult', {}).get('idlist', [])

            if pmids:
                pmid = pmids[0]
                # 获取摘要详情
                fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
                fetch_params = {
                    'db': 'pubmed',
                    'id': pmid,
                    'retmode': 'xml',
                    'rettype': 'abstract'
                }

                fetch_response = requests.get(fetch_url, params=fetch_params, headers=headers, timeout=8)
                if fetch_response.status_code == 200:
                    abstract = parse_pubmed_xml(fetch_response.text)
                    if abstract and is_valid_abstract(abstract):
                        return abstract

        return ""

    except Exception:
        return ""

def search_pubmed_by_title_fuzzy(title, max_results=3):
    """模糊匹配标题在PubMed中搜索"""
    if not title or len(title) < 10:
        return ""

    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        headers = {'User-Agent': 'AcademicResearchBot/1.0'}

        # 清理标题，移除特殊字符
        clean_title = re.sub(r'[^\w\s]', ' ', title)
        keywords = ' '.join(clean_title.split()[:6])  # 取前6个关键词

        search_params = {
            'db': 'pubmed',
            'term': f'({keywords})[Title]',
            'retmode': 'json',
            'retmax': max_results
        }

        response = requests.get(base_url, params=search_params, headers=headers, timeout=8)
        if response.status_code == 200:
            data = response.json()
            ids = data.get("esearchresult", {}).get("idlist", [])

            for pmid in ids:
                # 获取文章详情
                fetch_params = {
                    'db': 'pubmed',
                    'id': pmid,
                    'retmode': 'xml'
                }

                abs_response = requests.get(
                    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                    params=fetch_params,
                    headers=headers,
                    timeout=8
                )

                if abs_response.status_code == 200:
                    abs_text = parse_pubmed_xml(abs_response.text)
                    if abs_text:
                        # 用 difflib 判断标题相似度
                        xml_soup = BeautifulSoup(abs_response.text, "xml")
                        pub_title_elem = xml_soup.find("ArticleTitle")
                        if pub_title_elem:
                            pub_title = pub_title_elem.get_text()
                            score = difflib.SequenceMatcher(None, pub_title.lower(), title.lower()).ratio()

                            if score > 0.7:  # 相似度阈值
                                return abs_text
        return ""
    except Exception:
        return ""

def parse_pubmed_xml(xml_content):
    """解析PubMed XML获取摘要"""
    try:
        soup = BeautifulSoup(xml_content, 'xml')
        abstract = soup.find('AbstractText')
        if abstract:
            return abstract.get_text(strip=True, separator=' ')
        return ""
    except Exception:
        return ""

def convert_inverted_index_to_text(inverted_index):
    """将OpenAlex的倒排索引格式转换为普通文本"""
    try:
        if not inverted_index or not isinstance(inverted_index, dict):
            return ""

        position_word_map = {}
        for word, positions in inverted_index.items():
            for pos in positions:
                position_word_map[pos] = word

        sorted_positions = sorted(position_word_map.keys())
        text = ' '.join(position_word_map[pos] for pos in sorted_positions)
        return text
    except Exception:
        return ""

def clean_html(text):
    """清理HTML标签和多余空格"""
    if not text:
        return ""

    # 移除HTML标签
    text = re.sub(r'<[^>]+>', '', str(text))
    # 移除URL
    text = re.sub(r'http[s]?://\S+', '', text)
    # 标准化空格
    text = re.sub(r'\s+', ' ', text)
    # 移除过多换行
    text = re.sub(r'\n+', ' ', text)

    return text.strip()

def force_get_abstract(item):
    """强制获取摘要 - 主函数"""
    if not is_valid_research_paper(item):
        return "Not a research paper", "Filtered"

    doi = item.get("DOI", "")
    title = item.get("Title", "")
    original_abstract = item.get("Abstract_En", "")

    # 验证原始摘要质量
    if original_abstract and is_valid_abstract(original_abstract):
        return original_abstract, "Original"

    abstract = ""
    source = "Not Found"

    # 方法1: API获取（最快）
    if doi:
        abstract = get_abstract_from_apis(doi)
        if abstract:
            source = "API"

    # 方法2: PubMed DOI搜索
    if not abstract and doi:
        abstract = get_abstract_from_pubmed_by_doi(doi)
        if abstract:
            source = "PubMed"

    # 方法3: DOI网页提取
    if not abstract and doi:
        abstract = extract_abstract_from_doi_webpage(doi)
        if abstract:
            source = "DOI Webpage"

    # 方法4: PubMed标题模糊搜索
    if not abstract and title:
        abstract = search_pubmed_by_title_fuzzy(title)
        if abstract:
            source = "PubMed Fuzzy"

    # 最终fallback: 生成简单描述
    if not abstract and title:
        abstract = generate_basic_description(item)
        source = "Generated"

    return abstract, source

def generate_basic_description(item):
    """为没有摘要的文章生成基本描述"""
    title = item.get("Title", "")
    authors = item.get("Authors", "")
    journal = item.get("Journal", "")
    year = item.get("Year", "")

    parts = []
    if title:
        parts.append(f"This study investigates {title}.")
    if authors:
        first_author = authors.split(';')[0] if ';' in authors else authors.split(',')[0]
        parts.append(f"Authored by {first_author}.")
    if journal and year:
        parts.append(f"Published in {journal} ({year}).")
    elif year:
        parts.append(f"Published in {year}.")

    return " ".join(parts) if parts else "Abstract not available."

# 批量处理函数
def batch_force_get_abstracts(items, delay=0.5):
    """批量强制获取摘要"""
    results = []
    stats = {
        "total": len(items),
        "original": 0,
        "api": 0,
        "pubmed": 0,
        "doi_web": 0,
        "pubmed_fuzzy": 0,
        "generated": 0,
        "filtered": 0,
        "failed": 0
    }

    for i, item in enumerate(items):
        print(f"🔍 处理进度: {i+1}/{len(items)} - {item.get('Title', '')[:50]}...")

        abstract, source = force_get_abstract(item)

        # 更新统计
        if source == "Original":
            stats["original"] += 1
        elif source == "API":
            stats["api"] += 1
        elif source == "PubMed":
            stats["pubmed"] += 1
        elif source == "DOI Webpage":
            stats["doi_web"] += 1
        elif source == "PubMed Fuzzy":
            stats["pubmed_fuzzy"] += 1
        elif source == "Generated":
            stats["generated"] += 1
        elif source == "Filtered":
            stats["filtered"] += 1
        else:
            stats["failed"] += 1

        # 更新项目
        item["Abstract_En"] = abstract
        item["Abstract_Source"] = source
        results.append(item)

        # 添加延迟避免被ban
        if i < len(items) - 1:  # 最后一个不延迟
            time.sleep(delay)

    # 打印统计信息
    print(f"\n📊 摘要获取统计:")
    print(f"   原始摘要: {stats['original']}")
    print(f"   API获取: {stats['api']}")
    print(f"   PubMed: {stats['pubmed']}")
    print(f"   DOI网页: {stats['doi_web']}")
    print(f"   PubMed模糊: {stats['pubmed_fuzzy']}")
    print(f"   生成描述: {stats['generated']}")
    print(f"   过滤非论文: {stats['filtered']}")
    print(f"   获取失败: {stats['failed']}")
    print(f"   总体成功率: {(stats['total'] - stats['failed'] - stats['filtered']) / stats['total'] * 100:.1f}%")

    return results

print("✅ 高效版摘要获取功能加载完成！")

📄 加载高效版摘要获取功能...
✅ 高效版摘要获取功能加载完成！


# 7.PubMed搜索功能

In [79]:
# 单元格7：多源搜索功能 - PubMed (使用自定义搜索逻辑)
print("🔍 加载PubMed搜索功能 - 自定义搜索逻辑版...")

# ==================== 全局搜索配置 ====================
PUBMED_SEARCH_MODES = ["standard", "broad", "fuzzy"]  # 搜索模式
PUBMED_FALLBACK_STRATEGIES = ["and_search", "title_search", "abstract_search"]  # 备用策略
# ====================================================

def build_pubmed_query(mode="standard"):
    """构建PubMed查询 - 使用自定义搜索逻辑"""

    query = ""

    if LOGIC_OPERATOR == "CUSTOM":
        query = build_custom_query("pubmed")
    elif SEARCH_LOGIC.lower() == "or":
        query_terms = [f'"{keyword}"[Title/Abstract]' for keyword in KEYWORDS]
        query = " OR ".join(query_terms)
    elif SEARCH_LOGIC.lower() == "and":
        query_terms = [f'"{keyword}"[Title/Abstract]' for keyword in KEYWORDS]
        query = " AND ".join(query_terms)

    # 添加模式特定的修饰
    if mode == "broad": # 宽松模式，移除精确引号
        query = query.replace('"', '')
    elif mode == "fuzzy": # 模糊模式，添加通配符
        query = query.replace('[Title/Abstract]', '*[Title/Abstract]')

    # 添加时间范围
    year_query = f

🔍 加载PubMed搜索功能 - 自定义搜索逻辑版...


# 8.其他数据库搜索功能

In [80]:
# 单元格8：多源搜索功能 - 其他数据源 (使用自定义搜索逻辑)
print("🔍 加载其他数据源搜索功能...")
def search_crossref():
    """搜索CrossRef - 简化输出版本"""
    print("🔹 Searching CrossRef...")
    try:
        # 使用自定义搜索逻辑
        query = build_custom_query("crossref") # 修改为调用 build_custom_query
        print(f"   查询: {query}")

        url = "https://api.crossref.org/works"
        params = {
            "query": query,
            "filter": f"from-pub-date:{START_YEAR},until-pub-date:{END_YEAR}",
            "rows": MAX_RESULTS_PER_SOURCE
        }

        response = requests.get(url, params=params, timeout=30)

        if response.status_code != 200:
            print(f"   ❌ HTTP错误: {response.status_code}")
            return []

        data = response.json()
        items = []

        for i, item_data in enumerate(data.get("message", {}).get("items", [])):
            if i >= MAX_RESULTS_PER_SOURCE:
                break

            abs_text = clean_html(item_data.get("abstract", ""))
            authors = ", ".join([a.get("family", "") for a in item_data.get("author", [])]) if "author" in item_data else ""

            year = ""
            if item_data.get("created"):
                date_parts = item_data["created"].get("date-parts", [[None]])
                if date_parts and date_parts[0]:
                    year = str(date_parts[0][0]) if date_parts[0][0] else ""

            journal = ""
            container_title = item_data.get("container-title", [""])
            if container_title:
                journal = container_title[0]

            doi = item_data.get("DOI", "")

            item = {
                "ID": f"DOI:{doi}" if doi else f"CrossRef:{i}",
                "Title": item_data.get("title", [""])[0] if item_data.get("title") else "No title",
                "Authors": authors,
                "Year": year,
                "DOI": doi,
                "Link": doi_to_link(doi),
                "Abstract_En": abs_text,
                "Abstract_Source": "CrossRef" if abs_text else "No Abstract",
                "Source": "CrossRef",
                "Journal": journal,
                "Citation_Count": 0,
                "Impact_Factor": 0
            }

            if doi:
                citation_count = get_citation_count(item["Title"], authors, doi)
                item["Citation_Count"] = citation_count

            if journal:
                impact_factor = get_journal_impact_factor(journal)
                item["Impact_Factor"] = impact_factor

            items.append(item)

        print(f"   ✅ 找到 {len(items)} 个结果")
        return items

    except Exception as e:
        print(f"   ❌ 搜索失败: {e}")
        return []

def search_semantic_scholar():
    """搜索Semantic Scholar - 使用备用策略"""
    print("🔹 Searching Semantic Scholar...")

    try:
        # 方案1A: 使用更简单的查询
        query = f"{KEYWORDS[0]} {KEYWORDS[1]} {KEYWORDS[2]}"
        print(f"   📝 查询: {query}")

        # 尝试不同的API端点
        endpoints = [
            # 主要端点
            "https://api.semanticscholar.org/graph/v1/paper/search",
            # 备用端点
            "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
        ]

        for endpoint in endpoints:
            try:
                print(f"   🔄 尝试端点: {endpoint.split('/')[-1]}")

                params = {
                    "query": query,
                    "year": f"{START_YEAR}-{END_YEAR}",
                    "limit": min(MAX_RESULTS_PER_SOURCE, 10),  # 进一步减少数量
                    "fields": "title,abstract,authors,year,externalIds,url,venue,citationCount,paperId"
                }

                headers = {
                    'User-Agent': 'AcademicResearchTool/1.0',
                    'Accept': 'application/json'
                }

                time.sleep(2)  # 重要：增加延迟
                response = requests.get(endpoint, params=params, headers=headers, timeout=30)

                if response.status_code == 200:
                    data = response.json()
                    items = process_semantic_scholar_data(data)
                    print(f"   ✅ Semantic Scholar: 成功获取 {len(items)} 个结果")
                    return items
                elif response.status_code == 429:
                    print(f"   ⚠️ 端点 {endpoint.split('/')[-1]} 速率限制")
                    continue
                else:
                    print(f"   ❌ 端点 {endpoint.split('/')[-1]} 错误: {response.status_code}")
                    continue

            except Exception as e:
                print(f"   ❌ 端点 {endpoint.split('/')[-1]} 异常: {e}")
                continue

        # 如果所有端点都失败，尝试方案2：通过DOI搜索
        print("   🔄 尝试通过DOI搜索策略...")
        return search_semantic_scholar_by_doi()

    except Exception as e:
        print(f"   ❌ Semantic Scholar搜索失败: {e}")
        return []

def search_semantic_scholar_by_doi():
    """通过DOI在Semantic Scholar中搜索已知文章"""
    print("   🔄 通过DOI搜索已知文献...")

    # 已知的相关文献DOI列表（你可以扩展这个列表）
    known_dois = [
        "10.1016/j.jpsychires.2023.05.041",  # GLP-1和抑郁症的相关文献
        "10.1038/s42255-024-01113-9",        # GLP-1生理学
        "10.1016/j.jacbts.2018.09.004",      # GLP-1类似物
        "10.1016/j.cmet.2017.04.013",        # 肠道微生物群
        "10.1038/s42255-024-01068-x"         # 氢硫化物
    ]

    items = []
    for doi in known_dois:
        try:
            url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}"
            params = {
                "fields": "title,abstract,authors,year,externalIds,url,venue,citationCount,paperId"
            }

            time.sleep(1)  # 延迟避免限制
            response = requests.get(url, params=params, timeout=15)

            if response.status_code == 200:
                paper = response.json()
                items.append(create_semantic_scholar_item(paper, doi))
                print(f"      ✅ 通过DOI找到: {paper.get('title', 'Unknown')[:50]}...")
            else:
                print(f"      ❌ DOI {doi} 未找到")

        except Exception as e:
            print(f"      ❌ DOI {doi} 搜索失败: {e}")
            continue

    return items

def process_semantic_scholar_data(data):
    """处理Semantic Scholar返回的数据"""
    items = []
    papers = data.get("data", [])

    for i, paper in enumerate(papers):
        doi = paper.get("externalIds", {}).get("DOI", "")
        paper_id = paper.get("paperId", "")
        abs_text = clean_html(paper.get("abstract", ""))
        authors = ", ".join([a["name"] for a in paper.get("authors", [])]) if paper.get("authors") else ""

        year = str(paper.get("year", "")) if paper.get("year") else ""

        item = {
            "ID": f"SS:{paper_id}" if paper_id else f"SemanticScholar:{i}",
            "Title": paper.get("title", "No title"),
            "Authors": authors,
            "Year": year,
            "DOI": doi,
            "Link": doi_to_link(doi) if doi else paper.get("url", ""),
            "Abstract_En": abs_text,
            "Abstract_Source": "Semantic Scholar" if abs_text else "No Abstract",
            "Source": "Semantic Scholar",
            "Journal": paper.get("venue", ""),
            "Citation_Count": paper.get("citationCount", 0),
            "Impact_Factor": 0
        }

        journal = paper.get("venue", "")
        if journal:
            impact_factor = get_journal_impact_factor(journal)
            item["Impact_Factor"] = impact_factor

        items.append(item)

    return items

def create_semantic_scholar_item(paper, doi):
    """从单个论文数据创建项目"""
    paper_id = paper.get("paperId", "")
    abs_text = clean_html(paper.get("abstract", ""))
    authors = ", ".join([a["name"] for a in paper.get("authors", [])]) if paper.get("authors") else ""
    year = str(paper.get("year", "")) if paper.get("year") else ""

    return {
        "ID": f"SS:{paper_id}" if paper_id else f"SemanticScholar:DOI:{doi}",
        "Title": paper.get("title", "No title"),
        "Authors": authors,
        "Year": year,
        "DOI": doi,
        "Link": doi_to_link(doi) if doi else paper.get("url", ""),
        "Abstract_En": abs_text,
        "Abstract_Source": "Semantic Scholar" if abs_text else "No Abstract",
        "Source": "Semantic Scholar",
        "Journal": paper.get("venue", ""),
        "Citation_Count": paper.get("citationCount", 0),
        "Impact_Factor": 0
    }
def search_all_sources():
    """搜索所有数据源 - 增强反馈版本"""
    print("🚀 开始多源文献收集")
    print(f"🎯 搜索逻辑: {SEARCH_LOGIC}")
    print(f"📅 时间范围: {START_YEAR}-{END_YEAR}")
    print(f"📊 每源最大结果数: {MAX_RESULTS_PER_SOURCE}")

    all_items = []
    search_functions = [
        ("PubMed", search_pubmed),
        ("CrossRef", search_crossref),
        ("Semantic Scholar", search_semantic_scholar)
    ]

    total_start_time = time.time()

    for source_name, search_func in search_functions:
        try:
            print(f"\n{'='*50}")
            print(f"🔍 正在搜索 {source_name}...")
            source_start_time = time.time()

            results = search_func()
            source_time = time.time() - source_start_time

            all_items.extend(results)
            print(f"📥 从 {source_name} 添加了 {len(results)} 个结果 (耗时: {source_time:.1f}秒)")

            # 礼貌延迟
            if search_func != search_functions[-1][1]:  # 不是最后一个函数
                print("⏳ 等待1秒...")
                time.sleep(1)

        except Exception as e:
            print(f"⚠️ {source_name} 搜索错误: {e}")
            continue

    total_time = time.time() - total_start_time
    print(f"\n{'='*50}")
    print(f"🎉 多源搜索完成!")
    print(f"📊 总计找到 {len(all_items)} 篇文献")
    print(f"⏱️ 总耗时: {total_time:.1f}秒")
    print(f"📈 平均速度: {len(all_items)/total_time:.1f} 篇/秒" if total_time > 0 else "📈 速度: 极快")

    # 统计各来源数量
    source_stats = {}
    for item in all_items:
        source = item.get("Source", "Unknown")
        source_stats[source] = source_stats.get(source, 0) + 1

    print("📋 来源统计:")
    for source, count in source_stats.items():
        print(f"   {source}: {count}篇")

    return all_items

print("✅ 所有搜索功能加载完成！")
print(f"   搜索逻辑: {SEARCH_LOGIC}")
print(f"   时间范围: {START_YEAR}-{END_YEAR}")

🔍 加载其他数据源搜索功能...
✅ 所有搜索功能加载完成！
   搜索逻辑: or
   时间范围: 2000-2025


# 9.摘要补全和翻译

In [81]:
# 单元格9：摘要补全和翻译
print("🌐 加载摘要补全和翻译功能...")

def enhanced_abstract_completion(final_data):
    """增强的摘要补全"""
    print("\n🔍 增强摘要补全...")
    completion_stats = {"improved": 0, "unchanged": 0, "failed": 0}

    for i, item in enumerate(tqdm(final_data, desc="摘要补全")):
        original_abstract = item["Abstract_En"]

        if (not original_abstract or
            len(original_abstract) < 80 or
            original_abstract == "Abstract not available" or
            "not available" in original_abstract.lower()):

            new_abstract, new_source = force_get_abstract(item)
            item["Abstract_En"] = new_abstract
            item["Abstract_Source"] = new_source

            if (new_abstract != original_abstract and
                len(new_abstract) > len(original_abstract) and
                "not available" not in new_abstract.lower()):
                completion_stats["improved"] += 1
            else:
                completion_stats["unchanged"] += 1
        else:
            completion_stats["unchanged"] += 1

        # 翻译摘要
        if (item["Abstract_En"] and
            item["Abstract_En"] != "Abstract not available"):
            item["Abstract_Zh"] = translate_to_cn(item["Abstract_En"])
        else:
            item["Abstract_Zh"] = "未找到摘要"

    print(f"\n📊 摘要补全统计:")
    print(f"  改进的摘要: {completion_stats['improved']}")
    print(f"  未改变的摘要: {completion_stats['unchanged']}")

    return completion_stats

print("✅ 摘要补全和翻译功能加载完成！")

🌐 加载摘要补全和翻译功能...
✅ 摘要补全和翻译功能加载完成！


# 10.开始完整搜索流程

In [82]:
# 单元格10：执行完整搜索流程
print("🎯 开始执行完整搜索流程...")

# 阶段1: 多源搜索
print("\n📚 阶段1: 多源搜索")
all_items = search_all_sources()

if not all_items:
    print("❌ 没有找到文章。请检查搜索词。")
else:
    print(f"✅ 初始搜索完成: 从所有来源找到 {len(all_items)} 个项目")

    # 阶段2: 去重
    print("\n🔄 阶段2: 去重")
    final_data = advanced_deduplication(all_items)
    print(f"✅ 去重完成: {len(final_data)} 个唯一项目")

    # 阶段3: 摘要补全和翻译
    print("\n🌐 阶段3: 摘要补全和翻译")
    completion_stats = enhanced_abstract_completion(final_data)

    # 显示统计信息
    print(f"\n📊 最终统计:")
    print(f"  总唯一文章数: {len(final_data)}")

    # 来源统计
    source_counts = {}
    for article in final_data:
        source = article.get('Source', 'Unknown')
        source_counts[source] = source_counts.get(source, 0) + 1

    print(f"  来源分布:")
    for source, count in source_counts.items():
        print(f"    {source}: {count} 篇文章")

    print(f"  有DOI的文章: {len([a for a in final_data if a['DOI']])}")
    print(f"  有摘要的文章: {len([a for a in final_data if a['Abstract_En'] != 'Abstract not available'])}")

    total_citations = sum(a.get('Citation_Count', 0) for a in final_data)
    avg_citations = total_citations // len(final_data) if final_data else 0
    print(f"  总引用数: {total_citations}")
    print(f"  平均引用数: {avg_citations}")

    # 显示前5篇文章
    print(f"\n📋 前5篇文章预览:")
    for i, article in enumerate(final_data[:5], 1):
        print(f"  {i}. {article['Title'][:80]}...")
        print(f"     作者: {article['Authors'][:50]}...")
        print(f"     期刊: {article['Journal']}, 年份: {article['Year']}")
        print(f"     引用: {article['Citation_Count']}, 影响因子: {article['Impact_Factor']}")
        print()

    print("🎉 文献搜索完成！")
    print("💡 下一步: 你可以将 final_data 变量用于进一步分析或生成报告")

🎯 开始执行完整搜索流程...

📚 阶段1: 多源搜索
🚀 开始多源文献收集
🎯 搜索逻辑: or
📅 时间范围: 2000-2025
📊 每源最大结果数: 50

🔍 正在搜索 PubMed...
🔍 Searching PubMed...
   PubMed查询: None
   搜索逻辑: or
   🔄 发送PubMed API请求...
   ✅ 找到 0 篇文章，获取详细信息...
   ⚠️ PubMed未找到相关文章，尝试宽松搜索...
   🔄 尝试宽松搜索策略...
      尝试 AND搜索: ((GLP-1[Title/Abstract] AND depression[Title/Abstract]) OR mood[Title/Abstract]) AND (2000:2025[Date - Publication])
      ✅ AND搜索 找到 50 篇文章
      处理批次 1/1 (50篇)...
      解析 50 篇文章...
      ✅ 批次 1 处理完成: 50 篇文章
📥 从 PubMed 添加了 50 个结果 (耗时: 2.0秒)
⏳ 等待1秒...

🔍 正在搜索 CrossRef...
🔹 Searching CrossRef...
   查询: GLP-1 OR depression OR mood
   ✅ 找到 50 个结果
📥 从 CrossRef 添加了 50 个结果 (耗时: 5.9秒)
⏳ 等待1秒...

🔍 正在搜索 Semantic Scholar...
🔹 Searching Semantic Scholar...
   📝 查询: GLP-1 depression mood
   🔄 尝试端点: search
   ✅ Semantic Scholar: 成功获取 10 个结果
📥 从 Semantic Scholar 添加了 10 个结果 (耗时: 3.2秒)

🎉 多源搜索完成!
📊 总计找到 110 篇文献
⏱️ 总耗时: 13.1秒
📈 平均速度: 8.4 篇/秒
📋 来源统计:
   PubMed: 50篇
   CrossRef: 50篇
   Semantic Scholar: 10篇
✅ 初始搜索完成: 从所有来源找到 110 个项目

🔄 阶段2: 去重


摘要补全: 100%|██████████| 109/109 [03:40<00:00,  2.03s/it]


📊 摘要补全统计:
  改进的摘要: 37
  未改变的摘要: 72

📊 最终统计:
  总唯一文章数: 109
  来源分布:
    PubMed: 50 篇文章
    CrossRef: 50 篇文章
    Semantic Scholar: 9 篇文章
  有DOI的文章: 108
  有摘要的文章: 109
  总引用数: 682
  平均引用数: 6

📋 前5篇文章预览:
  1. Natural hallucinogens of fungal and animal origin: action and potentialapplicati...
     作者: Ciszowski Krzysztof, Ziaja Angelika, Niedzielska-A...
     期刊: Folia medica Cracoviensia, 年份: 2025
     引用: 0, 影响因子: 0

  2. Focus on mood disorders, obsessive-compulsive disorders, and prisoners' treatmen...
     作者: Serretti Alessandro...
     期刊: International clinical psychopharmacology, 年份: 2026
     引用: 0, 影响因子: 0

  3. An Open-Label, Single-Center Proof of Concept Study Evaluating the Efficacy and ...
     作者: Acosta-Madiedo Ana Sofia, Gutierrez Marcela, Gutie...
     期刊: Journal of drugs in dermatology : JDD, 年份: 2025
     引用: 0, 影响因子: 176.1

  4. Prospective association of device-based physical activity and sedentary time dur...
     作者: Werneck André O, Liang Chih-Sung, Smith Lee, Ald

# 11.数据预览和基本分析

In [83]:
# 单元格11：数据预览和基本分析
print("📊 数据预览和基本分析...")

if 'final_data' in locals() and final_data:
    # 转换为DataFrame以便分析
    import pandas as pd

    df = pd.DataFrame(final_data)

    print("📋 数据概览:")
    print(f"数据形状: {df.shape}")
    print("\n前5行数据:")
    display(df.head())

    print("\n📈 基本统计:")
    print(f"来源分布:")
    print(df['Source'].value_counts())

    print(f"\n年份分布:")
    year_counts = df['Year'].value_counts().sort_index()
    print(year_counts)

    print(f"\n引用数统计:")
    print(f"平均引用数: {df['Citation_Count'].mean():.1f}")
    print(f"最大引用数: {df['Citation_Count'].max()}")
    print(f"引用数大于10的文章: {(df['Citation_Count'] > 10).sum()}")

    print(f"\n影响因子统计:")
    high_impact = df[df['Impact_Factor'] > 10]
    print(f"高影响因子文章(IF>10): {len(high_impact)}篇")
    if len(high_impact) > 0:
        print("高影响因子期刊:")
        for journal in high_impact['Journal'].unique():
            print(f"  - {journal}")

    # 保存为CSV文件
    csv_filename = f"{DATE_PREFIX}_literature_results.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 数据已保存为: {csv_filename}")

else:
    print("❌ 没有可用的数据，请先运行搜索流程")

📊 数据预览和基本分析...
📋 数据概览:
数据形状: (109, 15)

前5行数据:


,Source,Title,Authors,Journal,Year,DOI,Abstract_En,Abstract_Source,URL,PMID,ID,Citation_Count,Impact_Factor,Abstract_Zh,Link
0,PubMed,Natural hallucinogens of fungal and animal ori...,"Ciszowski Krzysztof, Ziaja Angelika, Niedziels...",Folia medica Cracoviensia,2025,10.24425/fmc.2025.156119,Natural hallucinogens derived from fungi and a...,PubMed,https://pubmed.ncbi.nlm.nih.gov/41329968/,41329968,PMID:41329968,0,0.0,几个世纪以来，源自真菌和动物的天然致幻剂一直被用于不同文化的萨满教、仪式和医学实践中。这些化...,NaN
1,PubMed,"Focus on mood disorders, obsessive-compulsive ...",Serretti Alessandro,International clinical psychopharmacology,2026,10.1097/YIC.0000000000000611,This study investigates Focus on mood disorder...,Generated,https://pubmed.ncbi.nlm.nih.gov/41329871/,41329871,PMID:41329871,0,0.0,这项研究重点关注情绪障碍、强迫症和囚犯的治疗。作者：Serretti Alessandro。...,NaN
2,PubMed,"An Open-Label, Single-Center Proof of Concept ...","Acosta-Madiedo Ana Sofia, Gutierrez Marcela, G...",Journal of drugs in dermatology : JDD,2025,10.36849/JDD.9569,Hidradenitis suppurativa (HS) is a chronic inf...,PubMed,https://pubmed.ncbi.nlm.nih.gov/41329144/,41329144,PMID:41329144,0,176.1,化脓性汗腺炎 (HS) 是一种与肥胖和代谢失调相关的慢性炎症性疾病。目前的疗法产生不同的益处...,NaN
3,PubMed,Prospective association of device-based physic...,"Werneck André O, Liang Chih-Sung, Smith Lee, A...",Child and adolescent mental health,2025,10.1111/camh.70048,The present study aimed to analyze association...,PubMed,https://pubmed.ncbi.nlm.nih.gov/41328782/,41328782,PMID:41328782,0,0.0,本研究旨在分析设备测量的儿童时期体力活动和久坐时间与整个青春期心理健康相关结果之间的关联。使...,NaN
4,PubMed,Comorbidity of undiagnosed mood symptoms with ...,"Zhang Haoran, Liao Yingqi, Lin Zhiying, Wen Ha...",Epidemiology and psychiatric sciences,2025,10.1017/S2045796025100346,To investigate the association of midlife and ...,PubMed,https://pubmed.ncbi.nlm.nih.gov/41328607/,41328607,PMID:41328607,0,56.9,调查中年和晚年未确诊的情绪症状，特别是其合并症，与多地区和种族成年人的长期痴呆风险之间的关系...,NaN



📈 基本统计:
来源分布:
Source
PubMed              50
CrossRef            50
Semantic Scholar     9
Name: count, dtype: int64

年份分布:
Year
2002     1
2006     1
2011     1
2012     3
2013     3
2014     1
2015     4
2016    13
2018     3
2019     1
2020     3
2022     2
2023     2
2024    14
2025    55
2026     2
Name: count, dtype: int64

引用数统计:
平均引用数: 6.3
最大引用数: 215
引用数大于10的文章: 7

影响因子统计:
高影响因子文章(IF>10): 56篇
高影响因子期刊:
  - Journal of drugs in dermatology : JDD
  - Epidemiology and psychiatric sciences
  - Medicine
  - The journal of headache and pain
  - Journal of participatory medicine
  - Journal of psychiatric practice
  - International journal of surgical pathology
  - JAMA network open
  - Journal of cognitive neuroscience
  - Journal of speech, language, and hearing research : JSLHR
  - European journal of psychotraumatology
  - Journal of experimental orthopaedics
  - Neuroscience applied
  - Journal of rehabilitation and assistive technologies engineering
  - International journal of cl

# 12.HTML报告电脑版

In [84]:
# 单元格15：完整的HTML报告生成功能 - 修复版
print("📄 加载完整的HTML报告生成功能...")

from datetime import datetime

def generate_enhanced_html_report_with_toc(final_data):
    """生成带目录导航的增强HTML报告"""
    print("📄 生成带目录导航的增强HTML报告...")

    # 计算统计信息
    total_articles = len(final_data)
    articles_with_abstract = len([item for item in final_data if item.get("Abstract_En") and "not available" not in item.get("Abstract_En", "").lower()])
    articles_with_doi = len([item for item in final_data if item.get("DOI")])

    # 按年份统计
    year_stats = {}
    for item in final_data:
        year = item.get("Year", "Unknown")
        if year and year != "Unknown":
            year_str = str(year).strip()
            if year_str and year_str != "Unknown":
                if year_str in year_stats:
                    year_stats[year_str] += 1
                else:
                    year_stats[year_str] = 1

    # 按数据源统计
    source_stats = {}
    for item in final_data:
        source = item.get("Source", "Unknown")
        if source in source_stats:
            source_stats[source] += 1
        else:
            source_stats[source] = 1

    # 生成年份统计图表HTML
    year_chart_html = ""
    if year_stats:
        # 只处理可以转换为数字的年份
        numeric_years = {}
        for year, count in year_stats.items():
            try:
                year_int = int(year)
                if START_YEAR <= year_int <= END_YEAR:
                    numeric_years[year_int] = count
            except ValueError:
                continue

        if numeric_years:
            sorted_years = sorted(numeric_years.keys())
            max_count = max(numeric_years.values())

            for year in sorted_years:
                count = numeric_years[year]
                percentage = (count / max_count) * 80 if max_count > 0 else 0
                year_chart_html += f"""
                <div class="year-bar">
                    <div class="year-label">{year}</div>
                    <div class="bar-container">
                        <div class="bar-fill" style="width: {percentage}%"></div>
                        <div class="bar-count">{count}</div>
                    </div>
                </div>
                """

    # 生成目录
    toc_items = []
    for i, article in enumerate(final_data, 1):
        year = article.get("Year", "Unknown")
        journal = article.get("Journal", "")[:30]

        toc_item = f"""
        <li class="toc-item">
            <a href="#article-{i}" class="toc-link">
                <span class="toc-number">{i}.</span>
                <span class="toc-title">{article["Title"][:70]}...</span>
            </a>
            <div class="toc-meta">
                <span class="toc-year">{year}</span>
                <span class="toc-journal">{journal}</span>
            </div>
        </li>
        """
        toc_items.append(toc_item)

    # 生成完整的HTML内容
    html_content = f"""
    <!DOCTYPE html>
    <html lang="zh-CN">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>多源文献检索报告 - {', '.join(KEYWORDS)}</title>
        <style>
            :root {{
                --primary-color: #2c3e50;
                --secondary-color: #3498db;
                --accent-color: #e74c3c;
                --light-bg: #f8f9fa;
                --border-color: #ddd;
            }}

            * {{
                margin: 0;
                padding: 0;
                box-sizing: border-box;
            }}

            body {{
                font-family: 'Microsoft YaHei', 'Segoe UI', Arial, sans-serif;
                line-height: 1.6;
                color: #333;
                background: #f5f7fa;
                overflow-x: hidden;
            }}

            .container {{
                display: flex;
                min-height: 100vh;
                position: relative;
            }}

            /* 目录控制按钮 */
            .toc-toggle {{
                position: fixed;
                top: 20px;
                left: 20px;
                background: rgba(128, 128, 128, 0.3);
                backdrop-filter: blur(10px);
                -webkit-backdrop-filter: blur(10px);
                color: white;
                border: none;
                width: 40px;
                height: 40px;
                border-radius: 8px;
                cursor: pointer;
                font-size: 18px;
                z-index: 1001;
                box-shadow: 0 2px 10px rgba(0,0,0,0.2);
                display: flex;
                align-items: center;
                justify-content: center;
                border: 1px solid rgba(255, 255, 255, 0.2);
                line-height: 1;
                padding: 0;
                transition: all 0.3s ease;
            }}

            .toc-toggle:hover {{
                background: rgba(128, 128, 128, 0.5);
                backdrop-filter: blur(15px);
                -webkit-backdrop-filter: blur(15px);
                transform: translateY(-2px);
                box-shadow: 0 4px 15px rgba(0,0,0,0.3);
            }}

            /* 侧边栏样式 */
            .sidebar {{
                width: 350px;
                background: var(--primary-color);
                color: white;
                padding: 20px;
                position: fixed;
                top: 0;
                left: 0;
                height: 100vh;
                overflow-y: auto;
                border-right: 3px solid var(--secondary-color);
                transition: transform 0.3s ease;
                z-index: 1000;
            }}

            .sidebar.hidden {{
                transform: translateX(-100%);
            }}

            .sidebar-header {{
                text-align: center;
                margin-bottom: 25px;
                padding-bottom: 15px;
                border-bottom: 2px solid var(--secondary-color);
                margin-top: 20px;
            }}

            .sidebar-header h2 {{
                color: #ecf0f1;
                font-size: 1.3em;
                margin-bottom: 5px;
            }}

            .toc {{
                margin-top: 20px;
            }}

            .toc-list {{
                list-style: none;
                max-height: calc(100vh - 200px);
                overflow-y: auto;
            }}

            .toc-item {{
                margin-bottom: 12px;
                padding: 10px;
                border-radius: 8px;
                transition: all 0.3s ease;
                border-left: 3px solid transparent;
            }}

            .toc-item:hover {{
                background: rgba(255,255,255,0.1);
                border-left-color: var(--secondary-color);
            }}

            .toc-item.active {{
                background: rgba(52, 152, 219, 0.2);
                border-left-color: var(--secondary-color);
            }}

            .toc-link {{
                color: #ecf0f1;
                text-decoration: none;
                display: block;
                font-weight: 500;
            }}

            .toc-number {{
                color: var(--secondary-color);
                font-weight: bold;
                margin-right: 5px;
            }}

            .toc-title {{
                font-size: 0.9em;
                line-height: 1.4;
            }}

            .toc-meta {{
                margin-top: 5px;
                font-size: 0.75em;
                color: #bdc3c7;
                display: flex;
                justify-content: space-between;
            }}

            .toc-year {{
                background: rgba(52, 152, 219, 0.3);
                padding: 2px 6px;
                border-radius: 10px;
            }}

            .toc-journal {{
                font-style: italic;
            }}

            /* 主内容区样式 */
            .main-content {{
                flex: 1;
                padding: 30px;
                margin-left: 350px;
                transition: all 0.3s ease;
                min-height: 100vh;
            }}

            .main-content.expanded {{
                margin-left: 0;
            }}

            .header {{
                text-align: center;
                margin-bottom: 30px;
                padding: 25px;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                border-radius: 15px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.1);
                margin-top: 10px;
            }}

            .header h1 {{
                font-size: 2.2em;
                margin-bottom: 10px;
                text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
            }}

            .search-info {{
                background: var(--light-bg);
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 25px;
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px;
            }}

            .info-item {{
                text-align: center;
            }}

            .info-label {{
                font-size: 0.9em;
                color: #666;
                margin-bottom: 5px;
            }}

            .info-value {{
                font-size: 1.4em;
                font-weight: bold;
                color: var(--secondary-color);
            }}

            /* 年份统计样式 */
            .year-stats {{
                background: white;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 25px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}

            .year-stats h3 {{
                color: var(--primary-color);
                margin-bottom: 15px;
                text-align: center;
            }}

            .year-bar {{
                display: flex;
                align-items: center;
                margin-bottom: 10px;
                gap: 10px;
            }}

            .year-label {{
                width: 60px;
                font-weight: bold;
                color: var(--primary-color);
            }}

            .bar-container {{
                flex: 1;
                background: #ecf0f1;
                height: 25px;
                border-radius: 12px;
                position: relative;
                overflow: hidden;
            }}

            .bar-fill {{
                background: linear-gradient(90deg, #3498db, #2ecc71);
                height: 100%;
                border-radius: 12px;
                transition: width 0.5s ease;
            }}

            .bar-count {{
                position: absolute;
                right: 10px;
                top: 50%;
                transform: translateY(-50%);
                color: var(--primary-color);
                font-weight: bold;
                font-size: 0.8em;
            }}

            .sort-controls {{
                background: var(--light-bg);
                padding: 15px;
                border-radius: 8px;
                margin-bottom: 20px;
                text-align: center;
            }}

            .sort-btn {{
                background: var(--secondary-color);
                color: white;
                border: none;
                padding: 8px 15px;
                margin: 0 5px;
                border-radius: 5px;
                cursor: pointer;
                transition: background 0.3s;
            }}

            .sort-btn:hover {{
                background: #2980b9;
            }}

            .sort-btn.active {{
                background: var(--accent-color);
            }}

            /* 文章卡片样式 */
            .article {{
                border: 1px solid var(--border-color);
                border-radius: 12px;
                padding: 25px;
                margin-bottom: 25px;
                background: white;
                transition: all 0.3s ease;
                position: relative;
            }}

            .article:hover {{
                transform: translateY(-3px);
                box-shadow: 0 8px 25px rgba(0,0,0,0.15);
                border-left: 4px solid var(--secondary-color);
            }}

            .article-header {{
                display: flex;
                justify-content: space-between;
                align-items: flex-start;
                margin-bottom: 15px;
            }}

            .article-title {{
                color: var(--primary-color);
                font-size: 1.3em;
                font-weight: bold;
                margin-bottom: 10px;
                flex: 1;
                line-height: 1.4;
            }}

            .article-metrics {{
                display: flex;
                gap: 10px;
                margin-left: 15px;
                flex-wrap: wrap;
            }}

            .metric {{
                background: var(--light-bg);
                padding: 5px 10px;
                border-radius: 15px;
                font-size: 0.8em;
                display: flex;
                align-items: center;
                gap: 5px;
            }}

            .metric.citations {{
                background: #ffeaa7;
                color: #d35400;
            }}

            .metric.impact {{
                background: #a29bfe;
                color: white;
            }}

            .metric.year {{
                background: #74b9ff;
                color: white;
                padding: 5px 10px;
                border-radius: 15px;
                font-size: 0.8em;
                display: flex;
                align-items: center;
                gap: 5px;
            }}

            .article-meta {{
                color: #666;
                font-size: 0.95em;
                margin-bottom: 15px;
                line-height: 1.5;
            }}

            .doi-link {{
                color: var(--secondary-color);
                text-decoration: none;
                font-weight: 500;
            }}

            .doi-link:hover {{
                text-decoration: underline;
            }}

            .abstract-section {{
                margin-top: 20px;
            }}

            .abstract-en {{
                background: var(--light-bg);
                padding: 18px;
                border-radius: 8px;
                margin-bottom: 12px;
                line-height: 1.6;
            }}

            .abstract-zh {{
                background: #e8f4fd;
                padding: 18px;
                border-radius: 8px;
                border-left: 4px solid var(--secondary-color);
                line-height: 1.6;
            }}

            .abstract-source {{
                font-size: 0.8em;
                color: #7f8c8d;
                margin-top: 5px;
                text-align: right;
            }}

            /* 移动端遮罩层 */
            .sidebar-overlay {{
                display: none;
                position: fixed;
                top: 0;
                left: 0;
                right: 0;
                bottom: 0;
                background: rgba(0,0,0,0.5);
                z-index: 999;
            }}

            .sidebar-overlay.active {{
                display: block;
            }}

            /* 响应式设计 */
            @media (max-width: 1024px) {{
                .sidebar {{
                    width: 300px;
                    position: fixed;
                    left: 0;
                    top: 0;
                    height: 100vh;
                    transform: translateX(-100%);
                    z-index: 1000;
                }}

                .sidebar:not(.hidden) {{
                    transform: translateX(0);
                }}

                .main-content {{
                    margin-left: 0;
                    padding: 20px;
                    width: 100%;
                }}

                .toc-toggle {{
                    top: 15px;
                    left: 15px;
                }}

                .sidebar.hidden {{
                    transform: translateX(-100%);
                }}

                .sidebar-overlay.active {{
                    display: block;
                }}

                .header {{
                    margin-top: 20px;
                }}
            }}

            @media (max-width: 768px) {{
                .sidebar {{
                    width: 280px;
                }}

                .main-content {{
                    padding: 15px;
                }}

                .toc-toggle {{
                    top: 10px;
                    left: 10px;
                }}

                .article-header {{
                    flex-direction: column;
                }}

                .article-metrics {{
                    margin-left: 0;
                    margin-top: 10px;
                }}

                .search-info {{
                    grid-template-columns: 1fr;
                    gap: 10px;
                }}

                .header {{
                    padding: 20px;
                    margin-top: 15px;
                }}

                .header h1 {{
                    font-size: 1.8em;
                }}
            }}

            /* 桌面端样式 */
            @media (min-width: 1025px) {{
                .sidebar {{
                    transform: translateX(0);
                }}

                .sidebar-overlay {{
                    display: none !important;
                }}

                .toc-toggle {{
                    top: 20px;
                    left: 20px;
                }}
            }}
        </style>
    </head>
    <body>
        <!-- 移动端遮罩层 -->
        <div class="sidebar-overlay" id="sidebarOverlay"></div>

        <!-- 目录控制按钮 -->
        <button class="toc-toggle" id="tocToggle" onclick="toggleSidebar()">✕</button>

        <div class="container">
            <!-- 侧边栏 -->
            <div class="sidebar" id="sidebar">
                <div class="sidebar-header">
                    <h2>📑 文献目录</h2>
                    <div style="font-size: 0.9em; color: #bdc3c7;">共 {total_articles} 篇文献</div>
                </div>

                <div class="toc">
                    <ul class="toc-list">
                        {''.join(toc_items)}
                    </ul>
                </div>
            </div>

            <!-- 主内容区 -->
            <div class="main-content" id="mainContent">
                <div class="header">
                    <h1>📚 多源文献检索报告</h1>
                    <p>检索时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
                </div>

                <div class="search-info">
                    <div class="info-item">
                        <div class="info-label">关键词</div>
                        <div class="info-value">{", ".join(KEYWORDS)}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">时间范围</div>
                        <div class="info-value">{START_YEAR}-{END_YEAR}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">总文献数</div>
                        <div class="info-value">{total_articles}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">数据源</div>
                        <div class="info-value">{len(source_stats)} 个</div>
                    </div>
                </div>

                <!-- 年份统计 -->
                <div class="year-stats">
                    <h3>📊 文献发表年份分布</h3>
                    {year_chart_html if year_chart_html else "<p style='text-align: center; color: #666;'>暂无年份数据</p>"}
                </div>

                <div class="sort-controls">
                    <strong>排序方式: </strong>
                    <button class="sort-btn active" onclick="sortByCitations()">引用次数</button>
                    <button class="sort-btn" onclick="sortByImpact()">影响因子</button>
                    <button class="sort-btn" onclick="sortByYear()">发表年份</button>
                </div>
    """

    # 对文献进行排序：按引用次数降序
    sorted_articles = sorted(final_data, key=lambda x: x.get('Citation_Count', 0), reverse=True)

    # 文章内容
    for i, article in enumerate(sorted_articles, 1):
        # 安全访问字段
        doi_link = article.get('Link', '') if article.get('Link') else (f"https://doi.org/{article['DOI']}" if article.get('DOI') else "")
        link_text = article.get('DOI', '') if article.get('DOI') else "查看原文"

        # 构建指标标签
        metrics_html = ""
        if article.get('Citation_Count', 0) > 0:
            metrics_html += f'<div class="metric citations">📊 {article["Citation_Count"]} 引用</div>'

        impact_factor = article.get('Impact_Factor', 0)
        if impact_factor and impact_factor > 0:
            metrics_html += f'<div class="metric impact">⭐ IF: {impact_factor}</div>'

        # 添加年份
        year = article.get('Year', 'Unknown')
        if year and year != 'Unknown':
            metrics_html += f'<div class="metric year">📅 {year}</div>'

        # 安全访问摘要字段
        abstract_zh = article.get('Abstract_Zh', '暂无中文摘要')

        html_content += f"""
                <div class="article" id="article-{i}">
                    <div class="article-header">
                        <div class="article-title">
                            {i}. {article.get('Title', '无标题')}
                        </div>
                        <div class="article-metrics">
                            {metrics_html}
                        </div>
                    </div>

                    <div class="article-meta">
                        <strong>👥 作者:</strong> {article.get('Authors', '未知作者')}<br>
                        <strong>📅 年份:</strong> {article.get('Year', '未知年份')} |
                        <strong>📚 期刊:</strong> {article.get('Journal', '未知期刊')}<br>
                        <strong>🔗 链接:</strong>
                        <a href="{doi_link}" class="doi-link" target="_blank">{link_text}</a>
                        <span style="margin-left: 10px; color: #7f8c8d;">来源: {article.get('Source', '未知')} | ID: {article.get('ID', '未知')}</span>
                    </div>

                    <div class="abstract-section">
                        <strong>📄 英文摘要:</strong>
                        <div class="abstract-en">
                            {article.get('Abstract_En', '无摘要')}
                            <div class="abstract-source">
                                摘要来源: {article.get('Abstract_Source', '未知')}
                            </div>
                        </div>

                        <strong>📖 中文摘要:</strong>
                        <div class="abstract-zh">
                            {abstract_zh}
                        </div>
                    </div>
                </div>
        """

    # JavaScript功能 - 修复版
    html_content += """
            </div>
        </div>

        <script>
            // 目录显示/隐藏功能 - 修复版
            function toggleSidebar() {
                const sidebar = document.getElementById('sidebar');
                const mainContent = document.getElementById('mainContent');
                const toggleBtn = document.getElementById('tocToggle');
                const overlay = document.getElementById('sidebarOverlay');

                sidebar.classList.toggle('hidden');
                mainContent.classList.toggle('expanded');

                if (sidebar.classList.contains('hidden')) {
                    toggleBtn.textContent = '☰';
                    // 只在移动端显示遮罩层
                    if (window.innerWidth <= 1024) {
                        overlay.classList.remove('active');
                    }
                } else {
                    toggleBtn.textContent = '✕';
                    // 只在移动端显示遮罩层
                    if (window.innerWidth <= 1024) {
                        overlay.classList.add('active');
                    }
                }
            }

            // 点击遮罩层关闭目录
            document.getElementById('sidebarOverlay').addEventListener('click', function() {
                toggleSidebar();
            });

            // 排序功能
            function sortByCitations() {
                sortArticles('citations');
                setActiveButton(event.target);
            }

            function sortByImpact() {
                sortArticles('impact');
                setActiveButton(event.target);
            }

            function sortByYear() {
                sortArticles('year');
                setActiveButton(event.target);
            }

            function sortArticles(criteria) {
                const articlesContainer = document.querySelector('.main-content');
                const articles = Array.from(document.querySelectorAll('.article'));

                articles.sort((a, b) => {
                    let aValue, bValue;

                    switch(criteria) {
                        case 'citations':
                            aValue = parseInt(a.querySelector('.metric.citations')?.textContent.match(/\\d+/) || 0);
                            bValue = parseInt(b.querySelector('.metric.citations')?.textContent.match(/\\d+/) || 0);
                            return bValue - aValue;
                        case 'impact':
                            aValue = parseFloat(a.querySelector('.metric.impact')?.textContent.match(/IF: ([\\d.]+)/)?.[1] || 0);
                            bValue = parseFloat(b.querySelector('.metric.impact')?.textContent.match(/IF: ([\\d.]+)/)?.[1] || 0);
                            return bValue - aValue;
                        case 'year':
                            aValue = parseInt(a.querySelector('.article-meta').textContent.match(/年份: (\\d+)/)?.[1] || 0);
                            bValue = parseInt(b.querySelector('.article-meta').textContent.match(/年份: (\\d+)/)?.[1] || 0);
                            return bValue - aValue;
                        default:
                            return 0;
                    }
                });

                // 重新编号并插入
                articles.forEach((article, index) => {
                    const title = article.querySelector('.article-title');
                    title.innerHTML = title.innerHTML.replace(/^\\d+\\./, `${index + 1}.`);
                    articlesContainer.appendChild(article);
                });
            }

            function setActiveButton(activeButton) {
                document.querySelectorAll('.sort-btn').forEach(btn => {
                    btn.classList.remove('active');
                });
                activeButton.classList.add('active');
            }

            // 平滑滚动
            document.querySelectorAll('.toc-link').forEach(link => {
                link.addEventListener('click', function(e) {
                    e.preventDefault();
                    const targetId = this.getAttribute('href');
                    const targetElement = document.querySelector(targetId);
                    if (targetElement) {
                        targetElement.scrollIntoView({
                            behavior: 'smooth',
                            block: 'start'
                        });
                    }

                    // 移动端点击目录项后自动关闭目录
                    if (window.innerWidth <= 1024) {
                        toggleSidebar();
                    }
                });
            });

            // 滚动时高亮当前阅读的章节
            window.addEventListener('scroll', function() {
                const articles = document.querySelectorAll('.article');
                const tocItems = document.querySelectorAll('.toc-item');

                let current = '';
                articles.forEach(article => {
                    const articleTop = article.offsetTop;
                    const articleHeight = article.clientHeight;
                    if (pageYOffset >= (articleTop - 200)) {
                        current = article.getAttribute('id');
                    }
                });

                tocItems.forEach(item => {
                    item.classList.remove('active');
                    const link = item.querySelector('.toc-link');
                    if (link.getAttribute('href') === '#' + current) {
                        item.classList.add('active');
                    }
                });
            });

            // 窗口大小改变时重置状态
            window.addEventListener('resize', function() {
                const sidebar = document.getElementById('sidebar');
                const toggleBtn = document.getElementById('tocToggle');
                const overlay = document.getElementById('sidebarOverlay');
                const mainContent = document.getElementById('mainContent');

                if (window.innerWidth > 1024) {
                    // 桌面端：默认显示目录
                    sidebar.classList.remove('hidden');
                    toggleBtn.textContent = '✕';
                    overlay.classList.remove('active');
                    mainContent.classList.remove('expanded');
                } else {
                    // 移动端：默认隐藏目录
                    sidebar.classList.add('hidden');
                    toggleBtn.textContent = '☰';
                    mainContent.classList.add('expanded');
                }
            });

            // 初始化时设置正确状态
            document.addEventListener('DOMContentLoaded', function() {
                const toggleBtn = document.getElementById('tocToggle');
                const sidebar = document.getElementById('sidebar');
                const mainContent = document.getElementById('mainContent');

                if (window.innerWidth <= 1024) {
                    // 移动端：隐藏目录
                    sidebar.classList.add('hidden');
                    toggleBtn.textContent = '☰';
                    mainContent.classList.add('expanded');
                } else {
                    // 桌面端：显示目录
                    sidebar.classList.remove('hidden');
                    toggleBtn.textContent = '✕';
                    mainContent.classList.remove('expanded');
                }
            });
        </script>
    </body>
    </html>
    """

    return html_content

print("✅ HTML报告生成功能加载完成！已修复所有问题")

📄 加载完整的HTML报告生成功能...
✅ HTML报告生成功能加载完成！已修复所有问题


# 13.最终报告生成，修复自动下载

In [85]:
# 单元格16：生成最终报告（极简下载版）
print("🎯 生成最终HTML报告...")

if 'final_data' in locals() and final_data:
    # 生成HTML报告
    html_report = generate_enhanced_html_report_with_toc(final_data)

    # 保存HTML文件
    output_filename = f"{DATE_PREFIX}_Literature_Report.html"
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(html_report)

    print(f"✅ HTML报告已生成: {output_filename}")
    print(f"📁 文件大小: {os.path.getsize(output_filename) / 1024:.1f} KB")

    # 最简单直接的下载方法
    try:
        from google.colab import files
        files.download(output_filename)
        print("⬇️ 自动下载已启动...")
    except:
        # 如果在本地环境或其他环境
        from IPython.display import display, HTML
        import base64

        with open(output_filename, 'rb') as f:
            b64 = base64.b64encode(f.read()).decode()

        display(HTML(f'''
        <a href="data:text/html;base64,{b64}" download="{output_filename}">
            <button style="font-size:16px; padding:10px 20px;">⬇️ 点击下载报告</button>
        </a>
        '''))

    print("🎉 下载完成！")

else:
    print("❌ 没有可用的数据来生成报告")

🎯 生成最终HTML报告...
📄 生成带目录导航的增强HTML报告...
✅ HTML报告已生成: 3Dec2025_Literature_Report.html
📁 文件大小: 499.7 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 自动下载已启动...
🎉 下载完成！
